In [56]:
import os
import tensorflow as tf

from tensorflow.keras import datasets, layers, models, utils

In [57]:
def fasta_reader(directory):
    seqs = []
    for file in os.listdir(directory):
        seq = ''
        filepath = directory + file
        #print(file)
        f = open(filepath, 'r')
        if not file.startswith('.'):
            for i in f:
                if i[0] == '>':
                    if seq != '':
                        seqs.append(seq)
                        seq = ''
                else:
                    seq += i.strip()
            if seq != '':
                seqs.append(seq)
    return(seqs)

In [58]:
#for file in os.listdir('/Users/samklauer/GitHub/PawseySummer2024/segments/')[0:71]:
    #print(file)

In [59]:
segments = fasta_reader('/Users/samklauer/GitHub/PawseySummer2024/segments/')
nonseg = fasta_reader('/Users/samklauer/GitHub/PawseySummer2024/NCnonsegmentedsingle/')

In [60]:
len(segments)

4738

In [61]:
len(nonseg)

6569

In [62]:
encoding = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'T': [0, 0, 0, 1], 'U':[0, 0, 0, 1], 'N':[0,0,0,0]
    , 'M':[0,0,0,0], 'R':[0,0,0,0], 'W':[0,0,0,0], 'S':[0,0,0,0], 'Y':[0,0,0,0], 'K':[0,0,0,0], 'V':[0,0,0,0], 'H':[0,0,0,0], 'D':[0,0,0,0],
    'B':[0,0,0,0], 'E':[0,0,0,0], 'F':[0,0,0,0], 'I':[0,0,0,0], 'J':[0,0,0,0], 'L':[0,0,0,0], 'O':[0,0,0,0], 
    'P':[0,0,0,0], 'Q':[0,0,0,0], 'X':[0,0,0,0], 'Z':[0,0,0,0]}

In [63]:
ls = ['P', 'help']
for i in ls:
    if encoding.get(i):
        print('yep')

yep


In [64]:
for i in segments:
    if len(i) >20000:
        segments.remove(i)

In [65]:
def one_hot_encode(sequence):
    encoding = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'T': [0, 0, 0, 1], 'U':[0, 0, 0, 1], 'N':[0,0,0,0]
    , 'M':[0,0,0,0], 'R':[0,0,0,0], 'W':[0,0,0,0], 'S':[0,0,0,0], 'Y':[0,0,0,0], 'K':[0,0,0,0], 'V':[0,0,0,0], 'H':[0,0,0,0], 'D':[0,0,0,0],
    'B':[0,0,0,0], 'E':[0,0,0,0], 'F':[0,0,0,0], 'I':[0,0,0,0], 'J':[0,0,0,0], 'L':[0,0,0,0], 'O':[0,0,0,0], 
    'P':[0,0,0,0], 'Q':[0,0,0,0], 'X':[0,0,0,0], 'Z':[0,0,0,0]}
    encoded_sequence = []

    for nucleotide in sequence:
        if encoding.get(nucleotide):
            encoded_sequence.append(encoding.get(nucleotide))
        else:
            encoded_sequence.append([0,0,0,0])

    feature_length = 20000

    while len(encoded_sequence) <20000:
        encoded_sequence.append([0,0,0,0])

    return encoded_sequence

In [85]:
one_hot_non = []
for i in nonseg:
    one_hot_non.append(one_hot_encode(i))

In [86]:
one_hot_seg = []
for i in segments:
    one_hot_seg.append(one_hot_encode(i))

In [87]:
len(one_hot_seg)

500

In [88]:
len(one_hot_non)

500

In [89]:
y = [1] * len(one_hot_seg) + [0] * len(one_hot_non)

In [90]:
#y

In [91]:
X = one_hot_seg + one_hot_non

In [92]:
len(X)

1000

In [93]:
Z = []
for i in range(len(y)):
    Z.append(X[i])
    Z[i].append(y[i])

In [94]:
train = []
test = []
for i in range(int(.8*len(one_hot_seg))):
    train.append(Z[i])
for i in range(int(.8*len(one_hot_seg)), len(one_hot_seg)):
    test.append(Z[i])
for i in range(len(one_hot_seg), len(one_hot_seg) + int(.8 * len(one_hot_non))):
    train.append(Z[i])
for i in range(len(one_hot_seg) + int(.8 * len(one_hot_non)), len(Z)):
    test.append(Z[i])

In [95]:
len(train[0][0:-1])

20000

In [96]:
X_train = []
y_train = []
for i in train:
    X_train.append(i[0:-1])
    y_train.append(i[-1])

In [97]:
len(X_train)

800

In [98]:
X_test = []
y_test = []
for i in test:
    X_test.append(i[0:-1])
    y_test.append(i[-1])

In [99]:
model = models.Sequential()
model.add(layers.Conv1D(20, 2, activation='relu', input_shape=(20000,4)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(20, 2, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(20, 2, activation='relu'))
model.add(layers.MaxPooling1D())
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation="sigmoid"))

In [100]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_9 (Conv1D)           (None, 19999, 20)         180       
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 9999, 20)         0         
 1D)                                                             
                                                                 
 conv1d_10 (Conv1D)          (None, 9998, 20)          820       
                                                                 
 max_pooling1d_10 (MaxPoolin  (None, 4999, 20)         0         
 g1D)                                                            
                                                                 
 conv1d_11 (Conv1D)          (None, 4998, 20)          820       
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 2499, 20)        

In [101]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [102]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
25/25 [==============================] - 7s 256ms/step - loss: 0.6127 - accuracy: 0.6425
Epoch 2/10
25/25 [==============================] - 6s 256ms/step - loss: 0.5337 - accuracy: 0.7212
Epoch 3/10
25/25 [==============================] - 7s 260ms/step - loss: 0.5097 - accuracy: 0.7425
Epoch 4/10
25/25 [==============================] - 7s 261ms/step - loss: 0.4472 - accuracy: 0.7950
Epoch 5/10
25/25 [==============================] - 6s 257ms/step - loss: 0.4189 - accuracy: 0.7962
Epoch 6/10
25/25 [==============================] - 7s 260ms/step - loss: 0.3639 - accuracy: 0.8425
Epoch 7/10
25/25 [==============================] - 7s 263ms/step - loss: 0.3348 - accuracy: 0.8637
Epoch 8/10
25/25 [==============================] - 7s 261ms/step - loss: 0.2767 - accuracy: 0.9038
Epoch 9/10
25/25 [==============================] - 7s 263ms/step - loss: 0.2590 - accuracy: 0.9025
Epoch 10/10
25/25 [==============================] - 7s 273ms/step - loss: 0.2340 - accuracy: 0.9212

In [103]:
test_loss, test_acc = model.evaluate(X_test, y_test)

7/7 [==============================] - 1s 53ms/step - loss: 0.4602 - accuracy: 0.7800


In [53]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 19999, 20)         180       
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 9999, 20)         0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 9998, 20)          820       
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 4999, 20)         0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 4998, 20)          820       
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 2499, 20)        

In [54]:
batch_sizes = [64, 128, 256]
learning_rates = [1e-3, 1e-4, 1e-5]
epochs = [5, 10, 15]

In [55]:
ls5 = []
for i in batch_sizes:
    for j in learning_rates:
        for k in epochs:

            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = j),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
            
            model.fit(X_train, y_train, batch_size = i, epochs=k,shuffle = True)

            test_loss, test_acc = model.evaluate(X_test, y_test)

            ls5.append([i, j, test_loss, test_acc])

Epoch 1/5
2/2 [==============================] - 1s 155ms/step - loss: 0.3392 - accuracy: 0.8875
Epoch 2/5
2/2 [==============================] - 1s 166ms/step - loss: 0.3307 - accuracy: 0.8625
Epoch 3/5
2/2 [==============================] - 1s 154ms/step - loss: 0.2868 - accuracy: 0.9250
Epoch 4/5
2/2 [==============================] - 1s 150ms/step - loss: 0.2827 - accuracy: 0.9250
Epoch 5/5
1/1 [==============================] - 0s 206ms/step - loss: 0.8925 - accuracy: 0.6000
Epoch 1/10
2/2 [==============================] - 1s 155ms/step - loss: 0.2758 - accuracy: 0.8750
Epoch 2/10
2/2 [==============================] - 1s 153ms/step - loss: 0.2432 - accuracy: 0.9250
Epoch 3/10
2/2 [==============================] - 1s 165ms/step - loss: 0.2869 - accuracy: 0.8500
Epoch 4/10
2/2 [==============================] - 1s 152ms/step - loss: 0.2274 - accuracy: 0.9125
Epoch 5/10
2/2 [==============================] - 1s 149ms/step - loss: 0.2176 - accuracy: 0.9750
Epoch 6/10
2/2 [=========

KeyboardInterrupt: 

In [145]:
ls5

[[64, 0.001, 0.47184762358665466, 0.7250000238418579],
 [64, 0.001, 0.43870431184768677, 0.7749999761581421],
 [64, 0.001, 0.4028274416923523, 0.800000011920929],
 [64, 0.0001, 0.4029541015625, 0.800000011920929],
 [64, 0.0001, 0.39947694540023804, 0.824999988079071],
 [64, 0.0001, 0.3950316309928894, 0.8500000238418579],
 [64, 1e-05, 0.3925342261791229, 0.824999988079071],
 [64, 1e-05, 0.39212554693222046, 0.824999988079071],
 [64, 1e-05, 0.39355945587158203, 0.824999988079071],
 [128, 0.001, 0.6085519790649414, 0.75],
 [128, 0.001, 0.4092869758605957, 0.800000011920929],
 [128, 0.001, 0.38696563243865967, 0.824999988079071],
 [128, 0.0001, 0.40196818113327026, 0.800000011920929],
 [128, 0.0001, 0.4007951319217682, 0.875],
 [128, 0.0001, 0.4169038236141205, 0.875],
 [128, 1e-05, 0.41082820296287537, 0.875],
 [128, 1e-05, 0.40477389097213745, 0.8500000238418579]]